# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pickle
from scipy import stats
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
import numpy as np
import pandas as pd
import warnings
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
from itertools import chain
from pathlib import Path
from pypdf import PdfReader
import os
from matplotlib_venn import venn3, venn3_circles
import upsetplot
from sklearn.decomposition import PCA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE
import missingno as msno
from sklearn.cluster import DBSCAN, HDBSCAN

# Результаты анализов

## Load data from files

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium'

files = glob(f"{path}/Результаты анализов/files/*.pdf")
fns = []
for file in files:
    head, fn = os.path.split(file)
    fns.append(fn)

feats = pd.read_excel(f"{path}/Результаты анализов/features.xlsx")
if feats['prefix'].is_unique:
    feats.set_index('prefix', inplace=True)
else:
    raise ValueError(f"Features' prefixes are not unique!")
feats_same_line = feats.loc[feats['line'] == 0, :]
feats_diff_line = feats.loc[feats['line'] != 0, :]

feats_same_line_dict = dict(zip(feats_same_line.index.values, feats_same_line['feature'].values))
feats_diff_line_dict = dict(zip(feats_diff_line.index.values, feats_diff_line['feature'].values))

df = pd.DataFrame(columns=['Pages', 'File', '№ направления', 'ФИО', 'Дата рождения', 'Дата обследования', 'Пол'] + list(feats['feature'].unique()))

missed_pages = {}
missed_lines = []

with open(f"{path}/Результаты анализов/files/skip_starts.txt") as f:
    skip_starts = tuple(list(set(f.read().splitlines())))

for fn in fns:
    print(fn)
    
    missed_pages[fn] = []
    
    # reader = PdfReader(f"{path}/Результаты анализов/files/{fn}.pdf")
    reader = PdfReader(f"{path}/Результаты анализов/files/{fn}")
    
    for page_id, page in tqdm(enumerate(reader.pages)):
        lines = page.extract_text().splitlines()
        
        if lines[0] == 'Фамилия:':
            
            line_sex = 7
            line_birth = 4
            if lines[1] == 'Дата рождения:ЛПУ:':
                line_sex = 6
                line_birth = 3
            
            if lines[10].startswith('Дата:'):
                line_date = 10
                line_name = 12
            elif lines[9].startswith('Дата:'):
                line_date = 9
                line_name = 11
            else:
                raise ValueError(f"Wrong ID parsing: {fn} {page_id}")
                
            sample_name = lines[line_name].capitalize() + ' ' + re.findall(r"(.*)Имя", lines[line_name - 1])[0]
            sample_date = re.findall(r"Дата: (.*)", lines[line_date])[0]
            sample_number = lines[3]
            
            sample_id = f"{sample_name} {sample_date} {sample_number}"
            
            if sample_id in df.index:
                df.at[sample_id, 'Pages'] += f' {page_id}'
            else:
                df.at[sample_id, 'Pages'] = f'{page_id}'

            df.at[sample_id, 'File'] = fn
            df.at[sample_id, '№ направления'] = sample_number
            df.at[sample_id, 'ФИО'] = sample_name
            df.at[sample_id, 'Дата рождения'] = lines[line_birth].replace('17.03.7979', '17.03.1979')
            df.at[sample_id, 'Дата обследования'] = sample_date
            df.at[sample_id, 'Пол'] = re.findall(r"Пол: (.+)", lines[line_sex])[0][0]

            for line_id, line in enumerate(lines):
                line = line.replace("не обнаружено", "0.0")
                line = line.replace("\u2009", "")
                line = line.replace("в 1 мл", "в мл")
                if line in feats_diff_line_dict:
                    target_line = lines[line_id + feats_diff_line.at[line, 'line']]
                    line_parse = re.findall(fr"([-+]?(?:\d+\.\d+|\d+|\.\d+)).*", target_line)
                    df.at[sample_id, feats_diff_line_dict[line]] = line_parse[0]
                else:
                    line = line.replace(" - ", "-")
                    # line_parse_w_units = re.findall(fr"(.*\S)\s+([-+]?(?:\d+\.\d+|\d+|\.\d+)) (.*)", line)
                    # line_parse_wo_units = re.findall(fr"(.*\S)\s+([-+]?(?:\d+\.\d+|\d+|\.\d+)) ", line)
                    line_parse_w_units = re.findall(fr"(.*)\s([-+]?(?:\d+\.\d+|\d+|\.\d+)) (.*)", line)
                    line_parse_wo_units = re.findall(fr"(.*)\s([-+]?(?:\d+\.\d+|\d+|\.\d+)) ", line)
                    if len(line_parse_w_units) > 0:
                        if line_parse_w_units[0][0] in feats_same_line_dict:
                            feat_unit = feats_same_line.at[line_parse_w_units[0][0], 'unit']
                            if not pd.isna(feat_unit):
                                if feat_unit in line_parse_w_units[0][2] or feat_unit.replace('МЕ/', 'Ед/') in line_parse_w_units[0][2]:
                                    df.at[sample_id, feats_same_line_dict[line_parse_w_units[0][0]]] = line_parse_w_units[0][1]
                                else:
                                    print(f"{line} ({fn} {page_id} {line_id})")
                            else:
                                df.at[sample_id, feats_same_line_dict[line_parse_w_units[0][0]]] = line_parse_w_units[0][1]
                        else:
                            if not line.startswith(skip_starts):
                                missed_lines.append(f"{line} ({fn} {page_id} {line_id})")
                    elif len(line_parse_wo_units) > 0:
                        print(f"hello: {line_parse_wo_units}")
                        df.at[sample_id, feats_same_line_dict[line_parse_wo_units[0][0]]] = line_parse_wo_units[0][1]
        else:
            missed_pages[fn].append(page_id)

df = df.apply(pd.to_numeric, errors='ignore')
df['Дата рождения'] = pd.to_datetime(df['Дата рождения'], format="%d.%m.%Y").dt.date
df['Дата обследования'] = pd.to_datetime(df['Дата обследования'], format="%d.%m.%Y").dt.date
df.insert(7, 'Возраст', (df['Дата обследования'] - df['Дата рождения']) / np.timedelta64(1, 'D') / 365.25)
df.to_excel(f"{path}/Результаты анализов/data3.xlsx")

In [ ]:
reader = PdfReader(f"{path}/Результаты анализов/files/апр25.pdf")
print(reader.pages[933].extract_text())

In [ ]:
print('\n'.join(missed_lines))

In [ ]:
from collections import Counter
words = ' '.join(missed_lines).split(' ')
word_and_freq = dict(Counter(words).most_common())
df_word_and_freq = pd.Series(word_and_freq, name='Count').to_frame()
df_word_and_freq.to_excel(f"{path}/Результаты анализов/words.xlsx")

In [ ]:
print(missed_pages)

## Features precessing

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium'

df = pd.read_excel(f"{path}/Результаты анализов/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path}/Результаты анализов/features.xlsx", index_col='feature')
feats = feats[~feats.index.duplicated(keep='first')]

hist_bins = np.linspace(5, 115, 23)
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(6, 3.5))
histplot = sns.histplot(
    data=df,
    bins=hist_bins,
    edgecolor='k',
    linewidth=1,
    x="Возраст",
    color='crimson',
    ax=ax
)
histplot.set(xlim=(0, 120))
plt.savefig(f"{path}/Результаты анализов/hist_age_max_range.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Результаты анализов/hist_age_max_range.pdf", bbox_inches='tight')
plt.close(fig)

nan_pct = df[feats.index.to_list()].isna().sum().sum() / df.size * 100
print(f"% NaNs:{nan_pct}")

nan_feats = df[feats.index.to_list()].isna().sum(axis=0).to_frame(name="Number of NaNs")
feats.loc[feats.index, "Number of NaNs"] = nan_feats.loc[feats.index, "Number of NaNs"]
feats["% of NaNs"] = nan_feats["Number of NaNs"] / df.shape[0] * 100
feats["Number of not-NaNs"] = df[feats.index.to_list()].notna().sum(axis=0)
feats.sort_values(["% of NaNs"], ascending=[True], inplace=True)

sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(16, 4))
hist = sns.histplot(
    data=feats,
    x="% of NaNs",
    bins=np.linspace(-0.5, 100.5, 102),
    # discrete=True,
    edgecolor='k',
    linewidth=0.5,
    color='crimson',
    ax=ax
)
hist.set(xlim=(-0.5, 100.5))
hist.set_ylabel("Количество признаков")
hist.set_xlabel("% пропущенных значений")
plt.savefig(f"{path}/Результаты анализов/nan_feats_hist.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Результаты анализов/nan_feats_hist.pdf", bbox_inches='tight')
plt.close(fig)

feats[r"Pearson $\rho$"] = 0.0
for f in feats.index:
    df_tmp = df.loc[:, ['Возраст', f]].dropna(axis=0, how='any')
    if df_tmp.shape[0] > 1:
        if df_tmp[f].nunique() > 1:
            vals_1 = df_tmp.loc[:, 'Возраст'].values
            vals_2 = df_tmp.loc[:, f].values
            rho, _ = stats.pearsonr(vals_1, vals_2)
            feats.at[f, r"Pearson $\rho$"] = rho
        else:
            feats.at[f, r"Pearson $\rho$"] = 0.0

feats.to_excel(f"{path}/Результаты анализов/feats_with_metrics.xlsx", index_label="Features")

df_fig = feats.loc[feats['Number of not-NaNs'] >= 200, :]
df_fig['Features'] = df_fig.index
f_cmap = sns.color_palette("coolwarm", as_cmap=True)
# f_norm = mcolors.Normalize(vmin=min(df_fig[r"Pearson $\rho$"]), vmax=max(df_fig[r"Pearson $\rho$"])) 
f_norm = mcolors.TwoSlopeNorm(vcenter=0.0, vmin=min(df_fig[r"Pearson $\rho$"]), vmax=max(df_fig[r"Pearson $\rho$"]))
f_colors = {}
for cval in df_fig[r"Pearson $\rho$"]:
    f_colors.update({cval: f_cmap(f_norm(cval))})
    
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(4, 30), layout='constrained')
barplot = sns.barplot(
    data=df_fig,
    x='Number of not-NaNs',
    y='Features',
    hue=r"Pearson $\rho$",
    edgecolor='black',
    palette=f_colors,
    dodge=False,
    ax=ax
)
for container in barplot.containers:
    barplot.bar_label(container, label_type='edge', color='gray', fmt='%d', fontsize=8, padding=4.0)
ax.set_ylabel('')
ax.set(yticklabels=df_fig.index.to_list())
ax.get_legend().remove()
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
ax.set_xlabel('Количество записей not-NaN')
sm = plt.cm.ScalarMappable(cmap=f_cmap, norm=f_norm)
sm.set_array([])
cbar = barplot.figure.colorbar(sm, orientation="horizontal")
cbar.set_label("Корреляция с возрастом")
plt.savefig(f"{path}/Результаты анализов/feats_nans_and_age_correlation.pdf", bbox_inches='tight')
plt.savefig(f"{path}/Результаты анализов/feats_nans_and_age_correlation.png", bbox_inches='tight', dpi=200)
plt.close(fig)

## Select features sets

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium'

df = pd.read_excel(f"{path}/Результаты анализов/data.xlsx", index_col=0)

In [ ]:
feats = pd.read_excel(f"{path}/Результаты анализов/feats_with_metrics.xlsx", index_col=0)
feats_used = pd.read_excel(f"{path}/Результаты анализов/feats_used.xlsx", index_col=0)
feats_unused = feats.drop(feats_used.index.values, axis=0)

feats_cands = [
    'Общий белок, г/л',
    'Аспартатаминотрансфераза (АСТ), Ед/л',
    'Аланинаминотрансфераза (АЛТ), Ед/л',
    'Билирубин общий, мкмоль/л',
    'Холестерин общий, ммоль/л',
    'Глюкоза, ммоль/л',
    'Креатинин, мкмоль/л',
    
    'Железо, мкмоль/л',
    'Ферритин, мкг/л',

    'Холестерин липопротеидов высокой плотности (ЛПВП, HDL), ммоль/л',
    'Холестерин липопротеидов низкой плотности (ЛПНП, LDL), ммоль/л',
    'Триглицериды, ммоль/л',
    
    'Коэффициент атерогенности',
    'ЛПОНП, ммоль/л',
    'Гомоцистеин, мкмоль/л',
    'Мочевая кислота, мкмоль/л',
    'Билирубин прямой, мкмоль/л',
    'Витамин В9 (фолиевая кислота), нг/мл',
    'Витамин В12 (цианокобаламин), пг/мл',
    'Магний, ммоль/л',
    'Билирубин непрямой, мкмоль/л',
    'Кальций ионизированный, ммоль/л',
    'Гликированный гемоглобин А1с, %'
]

feats_cands = [
    'Тироксин свободный (Т4 свободный), пмоль/л',
    'Тиреотропный гормон (ТТГ), мкМЕ/мл',
    'Трийодтиронин свободный (Т3 свободный), пмоль/л',
    'Антитела к микросомальной тиреопероксидазе (Анти-ТПО), МЕ/мл',
    'Антитела к тиреоглобулину (Анти-ТГ), МЕ/мл',
    'Инсулин, мкМЕ/мл'
]

feats_cands = [
    'pH',
    'Билирубин, мг/дл',
    'Белок в моче, г/л',
    'Уробилиноген, мг/дл',
    'Кетоны, мг/дл',
    'Относительная плотность, г/мл',
    'Цилиндры зернистые, Ед/мкл',
    'Глюкоза в моче, ммоль/л',
    'Цилиндры гиалиновые, Ед/мкл',
    
    'Эпителий переходный, кл/мкл',
    'Эпителий плоский, кл/мкл',
    'Слизь, Ед/мкл',
    
    'Нитриты',
    'Эритроциты, кл/мкл',
    'Лейкоциты, кл/мкл'
]

feats_cands = [
    'Фолликулостимулирующий гормон (ФСГ), мМЕ/мл',
    'Лютеинизирующий гормон (ЛГ), мМЕ/мл',
    'Пролактин, мМЕ/л',
    'Тестостерон общий, нмоль/л',
    'Тестостерон свободный, нмоль/л',
    'Индекс свободных андрогенов, %',
    'ГСПГ (глобулин, связывающий половые гормоны), нмоль/л',
    'Простат-специфический антиген (ПСА) общий, нг/мл',
    'Простата-специфический антиген (ПСА) свободный, нг/мл',
    '% свободного ПСА'
]

feats.loc[feats_cands, :].to_excel(f"{path}/Результаты анализов/feats_new.xlsx")
feats_to_add = feats_unused.drop(feats_cands, axis=0).index.values
dict_n_rows = {}
for f in feats_to_add:
    dict_n_rows[f] = df.dropna(subset=feats_cands+[f]).shape[0]
df_n_rows = pd.Series(dict_n_rows, name='Count').to_frame()
df_n_rows.sort_values(["Count"], ascending=[False], inplace=True)
print(df.dropna(subset=feats_cands).shape[0])
print(df_n_rows.head(20))

# Результаты медасс

## Load data from files

In [ ]:
import logging

logger = logging.getLogger("pypdf")
logger.setLevel(logging.ERROR)


path = 'E:/YandexDisk/Work/bbd/millennium'

feats = pd.read_excel(f"{path}/Результаты медасс/features.xlsx", index_col=0)

df = pd.DataFrame(columns=['File'] + feats.index.to_list())

files = glob(f"{path}/Результаты медасс/files/*.pdf")

missed_files = {}

for file in (pbar := tqdm(files)):
    head, fn = os.path.split(file)
    pbar.set_description(f"{fn}")
    
    reader = PdfReader(f"{path}/Результаты медасс/files/{fn}", strict=False)
    pages = {}
    for p_id, p in enumerate(reader.pages):
        pages[p_id] = p.extract_text().splitlines()
    
    if pages[0][2] != 'Оценка состава тела (биоимпедансный анализ)' or len(pages) < 5:
        missed_files[fn] = len(pages)
    else:
        
        if 'Удельный основной обмен (ккал/м2/сут.)' in pages[0]:
            pages[0].remove('Удельный основной обмен (ккал/м2/сут.)')
        if 'Удельный основной обмен (ккал/м 2/сут.)' in pages[0]:
            pages[0].remove('Удельный основной обмен (ккал/м 2/сут.)')
        if 'по БЖМ' in pages[0]:
            pages[0].remove('по БЖМ')
        if pages[2][7].startswith('Ваш удельный основной обмен:'):
            pages[2].pop(7)
        skip_rows_page_4 = [
            'Индекс',
            'скелетно-мышечной',
            'массы (кг/м2)',
            'массы (кг/м 2)',
            '(кг)'
        ]
        for sr in skip_rows_page_4:
            if sr in pages[4]:
                pages[4].remove(sr)
        
        sample_name = re.sub(' +', ' ', pages[int(feats.at['ФИО', 'page'])][int(feats.at['ФИО', 'row'])]).rstrip()
        datetime_re_str = feats.at['Дата и время обследования', 're_string']
        sample_datetime = re.findall(fr"{datetime_re_str}", pages[int(feats.at['Дата и время обследования', 'page'])][int(feats.at['Дата и время обследования', 'row'])])[0]
        sample_id = f"{sample_name} {sample_datetime}"
        
        df.at[sample_id, 'File'] = fn
        
        for f in feats.index:
            
            f_str = re.sub(' +', ' ', pages[int(feats.at[f, 'page'])][int(feats.at[f, 'row'])]).rstrip()
            f_re_str = feats.at[f, 're_string']
            if not pd.isna(f_re_str):
                f_re_res = re.findall(fr"{f_re_str}", f_str)[0]
                f_re_group = feats.at[f, 're_group']
                if not pd.isna(f_re_group):
                    f_val = f_re_res[int(f_re_group)]
                else:
                    f_val = f_re_res
            else:
                f_val = f_str
            
            df.at[sample_id, f] = f_val

df = df.apply(pd.to_numeric, errors='ignore')
df['Дата и время обследования'] = pd.to_datetime(df['Дата и время обследования'], format="%d.%m.%Y %H:%M:%S")
print(f"Is index unique: {df.index.is_unique}")
df.to_excel(f"{path}/Результаты медасс/data2.xlsx")         

## Features processing

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium'

df = pd.read_excel(f"{path}/Результаты медасс/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path}/Результаты медасс/features.xlsx", index_col='feature')
feats = feats.loc[feats['input'] == 1]

hist_bins = np.linspace(5, 115, 23)
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(6, 3.5))
histplot = sns.histplot(
    data=df,
    bins=hist_bins,
    edgecolor='k',
    linewidth=1,
    x="Возраст",
    color='crimson',
    ax=ax
)
histplot.set(xlim=(0, 120))
plt.savefig(f"{path}/Результаты медасс/hist_age_max_range.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Результаты медасс/hist_age_max_range.pdf", bbox_inches='tight')
plt.close(fig)

nan_pct = df[feats.index.to_list()].isna().sum().sum() / df.size * 100
print(f"% NaNs:{nan_pct}")

nan_feats = df[feats.index.to_list()].isna().sum(axis=0).to_frame(name="Number of NaNs")
feats.loc[feats.index, "Number of NaNs"] = nan_feats.loc[feats.index, "Number of NaNs"]
feats["% of NaNs"] = nan_feats["Number of NaNs"] / df.shape[0] * 100
feats["Number of not-NaNs"] = df[feats.index.to_list()].notna().sum(axis=0)
feats.sort_values(["% of NaNs"], ascending=[True], inplace=True)

sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(16, 4))
hist = sns.histplot(
    data=feats,
    x="% of NaNs",
    bins=np.linspace(-0.5, 100.5, 102),
    # discrete=True,
    edgecolor='k',
    linewidth=0.5,
    color='crimson',
    ax=ax
)
hist.set(xlim=(-0.5, 100.5))
hist.set_ylabel("Количество признаков")
hist.set_xlabel("% пропущенных значений")
plt.savefig(f"{path}/Результаты медасс/nan_feats_hist.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Результаты медасс/nan_feats_hist.pdf", bbox_inches='tight')
plt.close(fig)

feats[r"Pearson $\rho$"] = 0.0
for f in feats.index:
    df_tmp = df.loc[:, ['Возраст', f]].dropna(axis=0, how='any')
    if df_tmp.shape[0] > 1:
        if df_tmp[f].nunique() > 1:
            vals_1 = df_tmp.loc[:, 'Возраст'].values
            vals_2 = df_tmp.loc[:, f].values
            rho, _ = stats.pearsonr(vals_1, vals_2)
            feats.at[f, r"Pearson $\rho$"] = rho
        else:
            feats.at[f, r"Pearson $\rho$"] = 0.0

feats.to_excel(f"{path}/Результаты медасс/feats_with_metrics.xlsx", index_label="Features")

df_fig = feats.loc[feats['Number of not-NaNs'] >= 100, :]
df_fig['Features'] = df_fig.index
f_cmap = sns.color_palette("coolwarm", as_cmap=True)
# f_norm = mcolors.Normalize(vmin=min(df_fig[r"Pearson $\rho$"]), vmax=max(df_fig[r"Pearson $\rho$"])) 
f_norm = mcolors.TwoSlopeNorm(vcenter=0.0, vmin=min(df_fig[r"Pearson $\rho$"]), vmax=max(df_fig[r"Pearson $\rho$"]))
f_colors = {}
for cval in df_fig[r"Pearson $\rho$"]:
    f_colors.update({cval: f_cmap(f_norm(cval))})
    
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(4, 16), layout='constrained')
barplot = sns.barplot(
    data=df_fig,
    x='Number of not-NaNs',
    y='Features',
    hue=r"Pearson $\rho$",
    edgecolor='black',
    palette=f_colors,
    dodge=False,
    ax=ax
)
for container in barplot.containers:
    barplot.bar_label(container, label_type='edge', color='gray', fmt='%d', fontsize=8, padding=4.0)
ax.set_ylabel('')
ax.set(yticklabels=df_fig.index.to_list())
ax.get_legend().remove()
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
ax.set_xlabel('Количество записей not-NaN')
sm = plt.cm.ScalarMappable(cmap=f_cmap, norm=f_norm)
sm.set_array([])
cbar = barplot.figure.colorbar(sm, orientation="horizontal")
cbar.set_label("Корреляция с возрастом")
plt.savefig(f"{path}/Результаты медасс/feats_nans_and_age_correlation.pdf", bbox_inches='tight')
plt.savefig(f"{path}/Результаты медасс/feats_nans_and_age_correlation.png", bbox_inches='tight', dpi=200)
plt.close(fig)

# Результаты экг

## Load data from files

In [ ]:
import logging

logger = logging.getLogger("pypdf")
logger.setLevel(logging.ERROR)


path = 'E:/YandexDisk/Work/bbd/millennium'

feats = pd.read_excel(f"{path}/Результаты экг/features.xlsx", index_col=0)

df = pd.DataFrame(columns=['File'] + feats.index.to_list())

files = glob(f"{path}/Результаты экг/files/*.pdf")

missed_files = []

for file in (pbar := tqdm(files)):
    head, fn = os.path.split(file)
    pbar.set_description(f"{fn}")
    
    reader = PdfReader(f"{path}/Результаты экг/files/{fn}", strict=False)
    
    pages = {0: reader.pages[0].extract_text().splitlines()}
    
    # Нет роста или веса
    if not any(x in pages[0][15] for x in ['см', 'кг']):
        pages[0] = pages[0][0:15] + [''] + pages[0][15:]
    # Лишняя строчка - есть ФИО врача    
    if (pages[0][17] != 'Дата/время:') and (pages[0][18] == 'Дата/время:'):
        pages[0].pop(17)
    # Нет номера 
    if pages[0][16] == 'Дата/время:':
        pages[0] = pages[0][0:16] + [''] + pages[0][16:]
        
    
    if pages[0][48] != 'КОМПЬЮТЕРНАЯ ИНТЕРПРЕТАЦИЯ ЭКГ' or pages[0][17] != 'Дата/время:':
        missed_files.append(f"{fn} initial")
    else:
    
        sample_name = re.sub(' +', ' ', pages[int(feats.at['ФИО', 'page'])][int(feats.at['ФИО', 'row'])]).rstrip()
        sample_number = re.sub(' +', ' ', pages[int(feats.at['Номер', 'page'])][int(feats.at['Номер', 'row'])]).rstrip()
        sample_datetime = re.sub(' +', ' ', pages[int(feats.at['Дата и время обследования', 'page'])][int(feats.at['Дата и время обследования', 'row'])]).rstrip()
        sample_id = f"{sample_name} {sample_number} {sample_datetime}"
        
        df.at[sample_id, 'File'] = fn
        
        for f in feats.index:
            
            f_str = re.sub(' +', ' ', pages[int(feats.at[f, 'page'])][int(feats.at[f, 'row'])]).rstrip()
            f_re_str = feats.at[f, 're_string']
            if not pd.isna(f_re_str):
                f_re_res = re.findall(fr"{f_re_str}", f_str)
                if len(f_re_res) > 0:
                    f_re_res = f_re_res[0]
                    f_re_group = feats.at[f, 're_group']
                    if not pd.isna(f_re_group):
                        f_val = f_re_res[int(f_re_group)]
                    else:
                        f_val = f_re_res
                else:
                    missed_files.append(f"{fn} {f}: {f_re_res}")
                    f_val = ''
            else:
                f_val = f_str
            
            df.at[sample_id, f] = f_val

df = df.apply(pd.to_numeric, errors='ignore')
df['Пол'] = df['Пол'].str.upper()
df['Дата и время обследования'] = pd.to_datetime(df['Дата и время обследования'], format="%d.%m.%Y %H:%M:%S")
df.to_excel(f"{path}/Результаты экг/data_raw_2.xlsx")

df_fixes = pd.read_excel(f"{path}/Результаты экг/data_fixes.xlsx", index_col=0)
df.update(df_fixes)
df.to_excel(f"{path}/Результаты экг/data_2.xlsx")

In [ ]:
missed_files

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium'

fn = #

reader = PdfReader(f"{path}/Результаты экг/files/{fn}.pdf")

print(reader.pages[0].extract_text())

## Features processing

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium'

df = pd.read_excel(f"{path}/Результаты экг/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path}/Результаты экг/features.xlsx", index_col='feature')
feats = feats.loc[feats['input'] == 1]

hist_bins = np.linspace(5, 115, 23)
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(6, 3.5))
histplot = sns.histplot(
    data=df,
    bins=hist_bins,
    edgecolor='k',
    linewidth=1,
    x="Возраст",
    color='crimson',
    ax=ax
)
histplot.set(xlim=(0, 120))
plt.savefig(f"{path}/Результаты экг/hist_age_max_range.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Результаты экг/hist_age_max_range.pdf", bbox_inches='tight')
plt.close(fig)

nan_pct = df[feats.index.to_list()].isna().sum().sum() / df.size * 100
print(f"% NaNs:{nan_pct}")

nan_feats = df[feats.index.to_list()].isna().sum(axis=0).to_frame(name="Number of NaNs")
feats.loc[feats.index, "Number of NaNs"] = nan_feats.loc[feats.index, "Number of NaNs"]
feats["% of NaNs"] = nan_feats["Number of NaNs"] / df.shape[0] * 100
feats["Number of not-NaNs"] = df[feats.index.to_list()].notna().sum(axis=0)
feats.sort_values(["% of NaNs"], ascending=[True], inplace=True)

sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(16, 4))
hist = sns.histplot(
    data=feats,
    x="% of NaNs",
    bins=np.linspace(-0.5, 100.5, 102),
    # discrete=True,
    edgecolor='k',
    linewidth=0.5,
    color='crimson',
    ax=ax
)
hist.set(xlim=(-0.5, 100.5))
hist.set_ylabel("Количество признаков")
hist.set_xlabel("% пропущенных значений")
plt.savefig(f"{path}/Результаты экг/nan_feats_hist.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/Результаты экг/nan_feats_hist.pdf", bbox_inches='tight')
plt.close(fig)

feats[r"Pearson $\rho$"] = 0.0
for f in feats.index:
    df_tmp = df.loc[:, ['Возраст', f]].dropna(axis=0, how='any')
    if df_tmp.shape[0] > 1:
        if df_tmp[f].nunique() > 1:
            vals_1 = df_tmp.loc[:, 'Возраст'].values
            vals_2 = df_tmp.loc[:, f].values
            rho, _ = stats.pearsonr(vals_1, vals_2)
            feats.at[f, r"Pearson $\rho$"] = rho
        else:
            feats.at[f, r"Pearson $\rho$"] = 0.0

feats.to_excel(f"{path}/Результаты экг/feats_with_metrics.xlsx", index_label="Features")

df_fig = feats.loc[feats['Number of not-NaNs'] >= 100, :]
df_fig['Features'] = df_fig.index
f_cmap = sns.color_palette("coolwarm", as_cmap=True)
# f_norm = mcolors.Normalize(vmin=min(df_fig[r"Pearson $\rho$"]), vmax=max(df_fig[r"Pearson $\rho$"])) 
f_norm = mcolors.TwoSlopeNorm(vcenter=0.0, vmin=min(df_fig[r"Pearson $\rho$"]), vmax=max(df_fig[r"Pearson $\rho$"]))
f_colors = {}
for cval in df_fig[r"Pearson $\rho$"]:
    f_colors.update({cval: f_cmap(f_norm(cval))})
    
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(4, 5), layout='constrained')
barplot = sns.barplot(
    data=df_fig,
    x='Number of not-NaNs',
    y='Features',
    hue=r"Pearson $\rho$",
    edgecolor='black',
    palette=f_colors,
    dodge=False,
    ax=ax
)
for container in barplot.containers:
    barplot.bar_label(container, label_type='edge', color='gray', fmt='%d', fontsize=8, padding=4.0)
ax.set_ylabel('')
ax.set(yticklabels=df_fig.index.to_list())
ax.get_legend().remove()
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
ax.set_xlabel('Количество записей not-NaN')
sm = plt.cm.ScalarMappable(cmap=f_cmap, norm=f_norm)
sm.set_array([])
cbar = barplot.figure.colorbar(sm, orientation="horizontal")
cbar.set_label("Корреляция с возрастом")
plt.savefig(f"{path}/Результаты экг/feats_nans_and_age_correlation.pdf", bbox_inches='tight')
plt.savefig(f"{path}/Результаты экг/feats_nans_and_age_correlation.png", bbox_inches='tight', dpi=200)
plt.close(fig)

# Samples intersections

In [ ]:
def get_sections(sets):
    """
    Given a list of sets, return a new list of sets with all the possible
    mutually exclusive overlapping combinations of those sets.  Another way
    to think of this is the mutually exclusive sections of a venn diagram
    of the sets.  If the original list has N sets, the returned list will
    have (2**N)-1 sets.

    Parameters
    ----------
    sets : list of set

    Returns
    -------
    combinations : list of tuple
        tag : str
            Binary string representing which sets are included / excluded in
            the combination.
        set : set
            The set formed by the overlapping input sets.
    """
    num_combinations = 2 ** len(sets)
    bit_flags = [2 ** n for n in range(len(sets))]
    flags_zip_sets = [z for z in zip(bit_flags, sets)]

    combo_sets = {}
    for bits in range(num_combinations - 1, 0, -1):
        include_sets = [s for flag, s in flags_zip_sets if bits & flag]
        exclude_sets = [s for flag, s in flags_zip_sets if not bits & flag]
        combo = set.intersection(*include_sets)
        combo = set.difference(combo, *exclude_sets)
        tag = ''.join([str(int((bits & flag) > 0)) for flag in bit_flags])
        combo_sets[tag] = combo
    return combo_sets

path = 'E:/YandexDisk/Work/bbd/millennium'

df_anl = pd.read_excel(f"{path}/Результаты анализов/data.xlsx", index_col=0)
df_bcp = pd.read_excel(f"{path}/Результаты медасс/data.xlsx", index_col=0)
df_ecg = pd.read_excel(f"{path}/Результаты экг/data.xlsx", index_col=0)

pathlib.Path(f"{path}/intersection").mkdir(parents=True, exist_ok=True)
sections = get_sections([set(df_anl['ФИО']), set(df_bcp['ФИО']), set(df_ecg['ФИО'])])
for sec in sections:
    df_sec = pd.DataFrame(index=list(sections[sec]))
    df_sec.to_excel(f"{path}/intersection/{sec}.xlsx", index_label='index')
    
fig, ax = plt.subplots()
venn = venn3(
    subsets=(set(df_anl['ФИО']), set(df_bcp['ФИО']), set(df_ecg['ФИО'])),
    set_labels = ('Анализы', 'Биоимпеданс', 'ЭКГ'),
    set_colors=('r', 'g', 'b'),
    alpha = 0.5)
venn3_circles(subsets=(set(df_anl['ФИО']), set(df_bcp['ФИО']), set(df_ecg['ФИО'])))
for text in venn.set_labels:
    text.set_fontsize(16)
for text in venn.subset_labels:
    text.set_fontsize(11)
    text.set_color('purple')
plt.savefig(f"{path}/intersection/venn3.png", bbox_inches='tight', dpi=400)
plt.savefig(f"{path}/intersection/venn3.pdf", bbox_inches='tight', dpi=400)
plt.clf()

dict_upset_lists = {
    'Анализы': list(set(df_anl['ФИО'])),
    '       Биоимпеданс': list(set(df_bcp['ФИО'])),
    'ЭКГ': list(set(df_ecg['ФИО']))
}
upset_all = list(set().union(*list(dict_upset_lists.values())))
df_upset = pd.DataFrame(index=upset_all)
for k, v in dict_upset_lists.items():
    df_upset[k] = df_upset.index.isin(v)
df_upset = df_upset.set_index(list(dict_upset_lists.keys()))
tmp = plt.figure(figsize=(9, 5))
upset_fig = upsetplot.UpSet(
    df_upset,
    sort_categories_by='input',
    subset_size='count',
    show_counts=True,
    min_degree=0,
    element_size=None,
    totals_plot_elements=3,
    include_empty_subsets=False
)
upset_fig.plot(tmp)
plt.savefig(f"{path}/intersection/upset.png", bbox_inches='tight')
plt.savefig(f"{path}/intersection/upset.pdf", bbox_inches='tight')
plt.close()

# Check features combinations shape

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium'
feats_set = 'Биохимические исследования'
df = pd.read_excel(f"{path}/Результаты анализов/data.xlsx", index_col=0)
df_feats = pd.read_excel(f"{path}/models/{feats_set}/feats.xlsx", index_col=0)

feats_in = df_feats.index.to_list()
feats_out = ['Возраст']
feats_add = ['ФИО', 'Пол', 'Дата рождения', 'Дата обследования']
feats_all = feats_add + feats_out + feats_in

with pd.ExcelWriter(f"{path}/models/{feats_set}/combinations.xlsx", engine='xlsxwriter') as writer:
    for feat_trgt in feats_in:
        print(feat_trgt)
        
        feats_selected = [feat_trgt]
        feats_remain = set(feats_in)
        feats_remain.remove(feat_trgt)
        
        df_stat = pd.DataFrame(columns=['shape'])
        
        while len(feats_remain) > 0:
            
            max_shape = 0
            max_feat = ''
            for feat_remain in feats_remain:
                curr_shape = df[feats_selected + [feat_remain]].dropna(axis=0, how='any').shape[0]
                if curr_shape > max_shape:
                    max_shape = curr_shape
                    max_feat = feat_remain
            df_stat.at[max_feat, 'shape'] = max_shape
            feats_selected.append(max_feat)
            feats_remain.remove(max_feat)
        
        df_stat.to_excel(writer, sheet_name=feat_trgt.replace('/', ' ')[0:30])

# Generate datasets

In [ ]:
colors = {
    'Гематологические исследования': 'crimson',
    'Щитовидная железа (6)': 'goldenrod',
    'Биохимические исследования (23)': 'fuchsia',
    'Половые гормоны (6)': 'forestgreen',
    'Половые гормоны': 'forestgreen',
    'Общий анализ мочи': 'lightblue',
    'Электрокардиограмма': 'lawngreen',
    'Оценка состава тела': 'deepskyblue',
}

data_path = {
    'Гематологические исследования': 'Результаты анализов',
    'Щитовидная железа (6)': 'Результаты анализов',
    'Биохимические исследования (23)': 'Результаты анализов',
    'Половые гормоны (6)': 'Результаты анализов',
    'Половые гормоны': 'Результаты анализов',
    'Общий анализ мочи': 'Результаты анализов',
    'Электрокардиограмма': 'Результаты экг',
    'Оценка состава тела': 'Результаты медасс',
}

path = 'E:/YandexDisk/Work/bbd/millennium'

feats_set = 'Половые гормоны'

df = pd.read_excel(f"{path}/{data_path[feats_set]}/data.xlsx", index_col=0)

df_feats = pd.read_excel(f"{path}/models/{feats_set}/feats.xlsx", index_col=0)
feats_in = df_feats.index.to_list()
feats_out = ['Возраст']
# feats_add = ['File', 'ФИО', 'Дата и время обследования', 'Пол']
feats_add = ['Pages', 'File', '№ направления', 'ФИО', 'Дата рождения', 'Дата обследования', 'Пол']
feats_all = feats_add + feats_out + feats_in

df = df[feats_all]

df_msno = df[feats_all].copy()
# df_msno.sort_values([feats_in[0]], ascending=[False], inplace=True)
msno_mtx = msno.matrix(
    df=df_msno,
    label_rotation=90,
    color=mcolors.to_rgb(colors[feats_set]),
    figsize=(0.7 * len(feats_in), 5),
)
plt.xticks(ha='center')
plt.setp(msno_mtx.xaxis.get_majorticklabels(), ha="center")
msno_mtx.set_title(feats_set, fontsize='large')
msno_mtx.set_ylabel("IDs", fontsize='large')
plt.savefig(f"{path}/models/{feats_set}/msno.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/models/{feats_set}/msno.pdf", bbox_inches='tight')
plt.clf()
df.dropna(axis=0, how='any', inplace=True)
df.to_excel(f"{path}/models/{feats_set}/data.xlsx", index_label="ID")
print(df.shape)

# Age histogramm
hist_bins = np.linspace(5, 115, 23)
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(6, 3.5), layout='constrained')
histplot = sns.histplot(
    data=df,
    bins=hist_bins,
    edgecolor='k',
    linewidth=1,
    x="Возраст",
    color=colors[feats_set],
    ax=ax
)
histplot.set(xlim=(0, 120))
histplot.set_ylabel('Количество')
histplot.set_title(feats_set)
plt.savefig(f"{path}/models/{feats_set}/age_hist.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/models/{feats_set}/age_hist.pdf", bbox_inches='tight')
plt.close(fig)

# Input features and output feature correlations
df_corr = pd.DataFrame(index=feats_in, columns=['rho'])
for f in tqdm(feats_in):
    df_tmp = df.loc[:, ['Возраст', f]].dropna(axis=0, how='any')
    if df_tmp.shape[0] > 1:
        vals_1 = df_tmp.loc[:, 'Возраст'].values
        vals_2 = df_tmp.loc[:, f].values
        df_corr.at[f, 'rho'], _ = stats.pearsonr(vals_1, vals_2)
df_corr.dropna(axis=0, how='any', inplace=True)
df_corr.insert(1, "abs(rho)", df_corr['rho'].abs())
df_corr.sort_values(["abs(rho)"], ascending=[False], inplace=True)
feats_cnt_wo_age = df_corr.index.to_list()
feats_cnt = ['Возраст'] + feats_cnt_wo_age
df_corr = df_corr.apply(pd.to_numeric)
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(0.8 + 0.038 * df_corr.index.str.len().max(), 0.9 + 0.4 * len(feats_cnt_wo_age) + 0.04 * df_corr.index.str.len().max()) , layout='constrained')
heatmap = sns.heatmap(
    df_corr.loc[:, ['rho']],
    annot=True,
    fmt=".2f",
    vmin=-1.0,
    vmax=1.0,
    cmap='coolwarm',
    linewidth=0.1,
    linecolor='black',
    #annot_kws={"fontsize": 15},
    cbar_kws={
        # "shrink": 0.9,
        # "aspect": 30,
        #'fraction': 0.046, 
        #'pad': 0.04,
    },
    ax=ax
)
heatmap_pos = ax.get_position()
ax.figure.axes[-1].set_position([heatmap_pos.x1 + 0.05, heatmap_pos.y0, 0.1, heatmap_pos.height])
ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$")
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_title(feats_set, fontsize=16)
ax.set(xticklabels=[])
ax.set(xticks=[])
plt.savefig(f"{path}/models/{feats_set}/age_feats_pearsonr.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/models/{feats_set}/age_feats_pearsonr.pdf", bbox_inches='tight')
plt.close(fig)

n_cols = 2
n_rows = int(np.ceil(len(feats_in) / n_cols))
n_empty = n_rows * n_cols - len(feats_in)
sns.set_theme(style='ticks')
fig, axs = plt.subplots(
    nrows=n_rows,
    ncols=n_cols,
    figsize=(n_cols * 3.0, n_rows * 2.5),
    gridspec_kw={'wspace':0.10, 'hspace': 0.05}, 
    sharex=True,
    layout='constrained'
)
for feat_id, feat in enumerate(df_corr.index.values):
    row_id, col_id = divmod(feat_id, n_cols)
    regplot = sns.regplot(
        data=df,
        x="Возраст",
        y=feat,
        color='crimson',
        scatter_kws=dict(
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=16,
        ),
        ax=axs[row_id, col_id]
    )
    axs[row_id, col_id].set_title(fr"Pearson $\rho$: {df_corr.loc[feat, 'rho']:0.3f}")
    y_labe_fontsize = min(15 / (len(feat) / 20), 13)
    axs[row_id, col_id].set_ylabel(feat, fontsize=y_labe_fontsize)
for empty_id in range(n_empty):   
    axs[n_rows - 1, n_cols - 1 - empty_id].axis('off')
fig.savefig(f"{path}/models/{feats_set}/age_feats_regplot.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/models/{feats_set}/age_feats_regplot.pdf", bbox_inches='tight')
plt.close(fig)

# Correlation heatmap
feats_cnt = ['Возраст'] + feats_in
df_corr = pd.DataFrame(data=np.zeros(shape=(len(feats_cnt), len(feats_cnt))), index=feats_cnt, columns=feats_cnt)
for f_id_1 in range(len(feats_cnt)):
    for f_id_2 in range(f_id_1, len(feats_cnt)):
        f_1 = feats_cnt[f_id_1]
        f_2 = feats_cnt[f_id_2]
        if f_id_1 != f_id_2:
            vals_1 = df.loc[:, f_1].values
            vals_2 = df.loc[:, f_2].values
            corr, pval = stats.pearsonr(vals_1, vals_2)
            df_corr.at[f_2, f_1] = pval
            df_corr.at[f_1, f_2] = corr
        else:
            df_corr.at[f_2, f_1] = np.nan
selection = np.tri(df_corr.shape[0], df_corr.shape[1], -1, dtype=bool)
df_fdr = df_corr.where(selection).stack().reset_index()
df_fdr.columns = ['row', 'col', 'pval']
_, df_fdr['pval_fdr_bh'], _, _ = multipletests(df_fdr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
nzmin = df_fdr['pval_fdr_bh'][df_fdr['pval_fdr_bh'].gt(0)].min(0) * 0.5
df_fdr['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
df_corr_fdr = df_corr.copy()
for line_id in range(df_fdr.shape[0]):
    df_corr_fdr.loc[df_fdr.at[line_id, 'row'], df_fdr.at[line_id, 'col']] = -np.log10(df_fdr.at[line_id, 'pval_fdr_bh'])
df_corr_fdr.to_excel(f"{path}/models/{feats_set}/feats_pearsonr.xlsx")
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(8.5 + 0.35 * len(feats_cnt), 6.5 + 0.25 * len(feats_cnt)), layout='constrained')
cmap_triu = plt.get_cmap("seismic").copy()
mask_triu=np.tri(len(feats_cnt), len(feats_cnt), -1, dtype=bool)
heatmap_diff = sns.heatmap(
    df_corr_fdr,
    mask=mask_triu,
    annot=True,
    fmt=".2f",
    center=0.0,
    cmap=cmap_triu,
    linewidth=0.1,
    linecolor='black',
    annot_kws={"fontsize": 32 / np.sqrt(len(df_corr_fdr.values) + 8)},
    ax=ax
)
ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$")
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
cmap_tril = plt.get_cmap("viridis").copy()
cmap_tril.set_under('black')
mask_tril=np.tri(len(feats_cnt), len(feats_cnt), -1, dtype=bool).T
heatmap_pval = sns.heatmap(
    df_corr_fdr,
    mask=mask_tril,
    annot=True,
    fmt=".1f",
    vmin=-np.log10(0.05),
    cmap=cmap_tril,
    linewidth=0.1,
    linecolor='black',
    annot_kws={"fontsize": 32 / np.sqrt(len(df_corr_fdr.values) + 8)},
    ax=ax
)
ax.figure.axes[-1].set_ylabel(r"$-\log_{10}(\mathrm{p-value})$")
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
ax.set_xlabel('', fontsize=16)
ax.set_ylabel('', fontsize=16)
ax.set_title(feats_set, fontsize=16)
plt.savefig(f"{path}/models/{feats_set}/feats_pearsonr.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/models/{feats_set}/feats_pearsonr.pdf", bbox_inches='tight')
plt.close(fig)

df_proc = df.copy()

# IQR outliers
feats_cnt = ['Возраст'] + feats_cnt_wo_age
out_columns = []
for f in tqdm(feats_cnt):
    q1 = df_proc[f].quantile(0.25)
    q3 = df_proc[f].quantile(0.75)
    iqr = q3 - q1
    df_proc[f"{f} IQR Outlier"] = 1
    out_columns.append(f"{f} IQR Outlier")
    filter = (df_proc[f] >= q1 - 1.5 * iqr) & (df_proc[f] <= q3 + 1.5 * iqr)
    df_proc.loc[filter, f"{f} IQR Outlier"] = 0
df_proc[f"Number of IQR Outliers"] = df_proc.loc[:, out_columns].sum(axis=1)

hist_bins = np.linspace(-0.5, len(feats_cnt) + 0.5, len(feats_cnt) + 2)
fig = plt.figure(figsize=(5, 3))
sns.set_theme(style='ticks')
histplot = sns.histplot(
    data=df_proc,
    x=f"Number of IQR Outliers",
    multiple="stack",
    bins=hist_bins,
    edgecolor='k',
    linewidth=1.0,
    color=colors[feats_set],
)
histplot.set(xlim=(-0.5, max(df_proc['Number of IQR Outliers'] + 0.5)))
histplot.set_title(feats_set)
histplot.set_xlabel("Количество IQR выбросов")
histplot.set_ylabel("Количество записей")
plt.savefig(f"{path}/models/{feats_set}/outs_iqr_hist.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/models/{feats_set}/outs_iqr_hist.pdf", bbox_inches='tight')
plt.close(fig)

out_columns = [f"{f} IQR Outlier" for f in feats_cnt]
df_msno = df_proc.loc[:, out_columns].copy()
df_msno.replace({1: np.nan}, inplace=True)
df_msno.rename(columns=dict(zip(out_columns, feats_cnt)), inplace=True)

# Plot barplot for features with outliers
msno_bar = msno.bar(
    df=df_msno,
    label_rotation=90,
    color=colors[feats_set],
    # figsize=(0.4 * len(feats_cnt), 4),
)
plt.xticks(ha='center')
plt.setp(msno_bar.xaxis.get_majorticklabels(), ha="center")
msno_bar.set_title(feats_set, fontsize='large')
msno_bar.set_ylabel("Записи без выбросов", fontsize='large')
plt.savefig(f"{path}/models/{feats_set}/outs_iqr_bar.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/models/{feats_set}/outs_iqr_bar.pdf", bbox_inches='tight')
plt.clf()

# Plot matrix of samples outliers distribution
msno_mtx = msno.matrix(
    df=df_msno,
    label_rotation=90,
    color=mcolors.to_rgb(colors[feats_set]),
    # figsize=(0.7 * len(feats_cnt), 5),
)
plt.xticks(ha='center')
plt.setp(msno_bar.xaxis.get_majorticklabels(), ha="center")
msno_mtx.set_title(feats_set, fontsize='large')
msno_mtx.set_ylabel("Записи", fontsize='large')
plt.savefig(f"{path}/models/{feats_set}/outs_iqr_matrix.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/models/{feats_set}/outs_iqr_matrix.pdf", bbox_inches='tight')
plt.clf()

# Plot heatmap of features outliers correlations
msno_heatmap = msno.heatmap(
    df=df_msno,
    label_rotation=90,
    cmap="bwr",
    fontsize=12,
    # figsize=(0.6 * len(feats_cnt), 0.6 * len(feats_cnt))
)
msno_heatmap.set_title(feats_set, fontsize='large')
plt.setp(msno_heatmap.xaxis.get_majorticklabels(), ha="center")
msno_heatmap.collections[0].colorbar.ax.tick_params(labelsize=20)
plt.savefig(f"{path}/models/{feats_set}/outs_iqr_heatmap.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/models/{feats_set}/outs_iqr_heatmap.pdf", bbox_inches='tight')
plt.clf()
    
# Dimensionality reduction
feats_cnt = ['Возраст'] + feats_in
dim_red_models = {
    't-SNE': TSNE(n_components=2),
    'PCA': PCA(n_components=2, whiten=False),
    'IsoMap': Isomap(n_components=2, n_neighbors=5),
    'MDS': MDS(n_components=2, metric=True),
    'GRP': GaussianRandomProjection(n_components=2, eps=0.5),
    'SRP': SparseRandomProjection(n_components=2, density='auto', eps=0.5, dense_output=False),
}
feats_dim_red = []
for drm in dim_red_models:
    dim_red_res = dim_red_models[drm].fit_transform(df_proc.loc[:, feats_cnt].values)
    df_proc.loc[:, f"{drm} 1"] = dim_red_res[:, 0]
    df_proc.loc[:, f"{drm} 2"] = dim_red_res[:, 1]
    df_proc.loc[:, f"{drm} HDBSCAN"] = HDBSCAN(min_cluster_size=int(df_proc.shape[0] * 0.05)).fit(df_proc.loc[:, [f"{drm} 1", f"{drm} 2"]].values).labels_
    feats_dim_red += [ f"{drm} 1",  f"{drm} 2"]
n_rows = 2
n_cols = 3
fig_height = 10
fig_width = 15
sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharey=False, sharex=False, layout='constrained')
for drm_id, drm in enumerate(dim_red_models.keys()):
    row_id, col_id = divmod(drm_id, n_cols)
    scatter = sns.scatterplot(
        data=df_proc,
        x=f"{drm} 1",
        y=f"{drm} 2",
        # hue=f"{drm} HDBSCAN",
        hue='Пол',
        palette={'М': 'deepskyblue', 'Ж': 'hotpink'},
        linewidth=0.25,
        alpha=0.75,
        edgecolor="k",
        s=40,
        # color=colors[feats_set],
        ax=axs[row_id, col_id],
    )
    axs[row_id, col_id].set_title(drm)
    # axs[n_rows - 1, n_cols - 1].axis('off')
fig.suptitle(feats_set, fontsize='large')   
fig.savefig(f"{path}/models/{feats_set}/dim_red.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/models/{feats_set}/dim_red.pdf", bbox_inches='tight')
df_proc.to_excel(f"{path}/models/{feats_set}/df_proc.xlsx", index_label="ID")
plt.close(fig)


## Replot figures

In [ ]:
colors = {
    'Гематологические исследования': 'crimson',
    'Гормоны': 'goldenrod',
    'Биохимические исследования 1': 'fuchsia',
    'Биохимические исследования 2': 'indianred',
    'Электрокардиограмма': 'lawngreen',
    'Оценка состава тела': 'deepskyblue'
}

data_path = {
    'Гематологические исследования': 'Результаты анализов',
    'Гормоны': 'Результаты анализов',
    'Биохимические исследования 1': 'Результаты анализов',
    'Биохимические исследования 2': 'Результаты анализов',
    'Электрокардиограмма': 'Результаты экг',
    'Оценка состава тела': 'Результаты медасс',
}

suffix = '' #'basic_plus_few_suspicious_23'

path = 'E:/YandexDisk/Work/bbd/millennium'

feats_set = 'Электрокардиограмма'

df = pd.read_excel(f"{path}/models/{feats_set}/data.xlsx", index_col=0)

df_feats = pd.read_excel(f"{path}/models/{feats_set}/feats.xlsx", index_col=0)
feats_in = df_feats.index.to_list()
feats_out = ['Возраст']
feats_add = ['ФИО', 'Пол']
feats_all = feats_add + feats_out + feats_in

# Input features and output feature correlations
df_corr = pd.DataFrame(index=feats_in, columns=['rho'])
for f in tqdm(feats_in):
    df_tmp = df.loc[:, ['Возраст', f]].dropna(axis=0, how='any')
    if df_tmp.shape[0] > 1:
        vals_1 = df_tmp.loc[:, 'Возраст'].values
        vals_2 = df_tmp.loc[:, f].values
        df_corr.at[f, 'rho'], _ = stats.pearsonr(vals_1, vals_2)
df_corr.dropna(axis=0, how='any', inplace=True)
df_corr.insert(1, "abs(rho)", df_corr['rho'].abs())
df_corr.sort_values(["abs(rho)"], ascending=[False], inplace=True)
feats_cnt_wo_age = df_corr.index.to_list()
feats_cnt = ['Возраст'] + feats_cnt_wo_age
df_corr = df_corr.apply(pd.to_numeric)
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(0.4 + 0.05 * df_corr.index.str.len().max(), 0.5 + 0.4 * len(feats_cnt_wo_age) + 0.04 * df_corr.index.str.len().max()) , layout='constrained')
heatmap = sns.heatmap(
    df_corr.loc[:, ['rho']],
    annot=True,
    fmt=".2f",
    vmin=-1.0,
    vmax=1.0,
    cmap='coolwarm',
    linewidth=0.1,
    linecolor='black',
    #annot_kws={"fontsize": 15},
    cbar_kws={
        # "shrink": 0.9,
        # "aspect": 30,
        #'fraction': 0.046, 
        #'pad': 0.04,
    },
    ax=ax
)
heatmap_pos = ax.get_position()
ax.figure.axes[-1].set_position([heatmap_pos.x1 + 0.05, heatmap_pos.y0, 0.1, heatmap_pos.height])
ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$")
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_title(feats_set, fontsize=16)
ax.set(xticklabels=[])
ax.set(xticks=[])
if suffix != '':
    plt.savefig(f"{path}/models/{feats_set}/pytorch_tabular/{suffix}/age_feats_pearsonr.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/models/{feats_set}/pytorch_tabular/{suffix}/age_feats_pearsonr.pdf", bbox_inches='tight')
else:
    plt.savefig(f"{path}/models/{feats_set}/pytorch_tabular/age_feats_pearsonr.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/models/{feats_set}/pytorch_tabular/age_feats_pearsonr.pdf", bbox_inches='tight')
plt.close(fig)

# Correlation heatmap
feats_cnt = ['Возраст'] + feats_in
df_corr = pd.DataFrame(data=np.zeros(shape=(len(feats_cnt), len(feats_cnt))), index=feats_cnt, columns=feats_cnt)
for f_id_1 in range(len(feats_cnt)):
    for f_id_2 in range(f_id_1, len(feats_cnt)):
        f_1 = feats_cnt[f_id_1]
        f_2 = feats_cnt[f_id_2]
        if f_id_1 != f_id_2:
            vals_1 = df.loc[:, f_1].values
            vals_2 = df.loc[:, f_2].values
            corr, pval = stats.pearsonr(vals_1, vals_2)
            df_corr.at[f_2, f_1] = pval
            df_corr.at[f_1, f_2] = corr
        else:
            df_corr.at[f_2, f_1] = np.nan
selection = np.tri(df_corr.shape[0], df_corr.shape[1], -1, dtype=bool)
df_fdr = df_corr.where(selection).stack().reset_index()
df_fdr.columns = ['row', 'col', 'pval']
_, df_fdr['pval_fdr_bh'], _, _ = multipletests(df_fdr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
nzmin = df_fdr['pval_fdr_bh'][df_fdr['pval_fdr_bh'].gt(0)].min(0) * 0.5
df_fdr['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
df_corr_fdr = df_corr.copy()
for line_id in range(df_fdr.shape[0]):
    df_corr_fdr.loc[df_fdr.at[line_id, 'row'], df_fdr.at[line_id, 'col']] = -np.log10(df_fdr.at[line_id, 'pval_fdr_bh'])
if suffix != '':
    df_corr_fdr.to_excel(f"{path}/models/{feats_set}/pytorch_tabular/{suffix}/feats_pearsonr.xlsx")
else:
    df_corr_fdr.to_excel(f"{path}/models/{feats_set}/pytorch_tabular/feats_pearsonr.xlsx")
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(8.5 + 0.35 * len(feats_cnt), 6.5 + 0.25 * len(feats_cnt)), layout='constrained')
cmap_triu = plt.get_cmap("seismic").copy()
mask_triu=np.tri(len(feats_cnt), len(feats_cnt), -1, dtype=bool)
heatmap_diff = sns.heatmap(
    df_corr_fdr,
    mask=mask_triu,
    annot=True,
    fmt=".2f",
    center=0.0,
    cmap=cmap_triu,
    linewidth=0.1,
    linecolor='black',
    annot_kws={"fontsize": 32 / np.sqrt(len(df_corr_fdr.values) + 8)},
    ax=ax
)
ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$")
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
cmap_tril = plt.get_cmap("viridis").copy()
cmap_tril.set_under('black')
mask_tril=np.tri(len(feats_cnt), len(feats_cnt), -1, dtype=bool).T
heatmap_pval = sns.heatmap(
    df_corr_fdr,
    mask=mask_tril,
    annot=True,
    fmt=".1f",
    vmin=-np.log10(0.05),
    cmap=cmap_tril,
    linewidth=0.1,
    linecolor='black',
    annot_kws={"fontsize": 32 / np.sqrt(len(df_corr_fdr.values) + 8)},
    ax=ax
)
ax.figure.axes[-1].set_ylabel(r"$-\log_{10}(\mathrm{p-value})$")
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
ax.set_xlabel('', fontsize=16)
ax.set_ylabel('', fontsize=16)
ax.set_title(feats_set, fontsize=16)
if suffix != '':
    plt.savefig(f"{path}/models/{feats_set}/pytorch_tabular/{suffix}/feats_pearsonr.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/models/{feats_set}/pytorch_tabular/{suffix}/feats_pearsonr.pdf", bbox_inches='tight')
else:
    plt.savefig(f"{path}/models/{feats_set}/pytorch_tabular/feats_pearsonr.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/models/{feats_set}/pytorch_tabular/feats_pearsonr.pdf", bbox_inches='tight')
plt.close(fig)

# Legacy

## Body composition

In [ ]:
feats_rows_bia = {
    'ID': 3,
    'Дата и время обследования': 4,
    'Возраст': 5,
    'Пол': 5,
    'Рост': 6,
    'Вес': 6,
    'Окр. талии, см': 7,
    'Окр. бедер, см': 7,
    'Сопрот. акт. на 5 кГц, Ом': 12,
    'Сопрот. акт. на 50 кГц, Ом': 12,
    'Сопрот. реакт. на 50 кГц, Ом': 12,
    'Фазовый угол (50 кГц), град.': 13,
    'Клеточная жидкость, кг': 14,
    'Удельный основной обмен, ккал/м²/сут.': 15,
    'Индекс массы тела': 18,
    'Индекс массы тела (% от середины нормы)': 21,
    'Жировая масса, кг': 23,
    'Жировая масса, кг (% от середины нормы)': 26,
    'Тощая масса, кг': 28,
    'Тощая масса, кг (% от середины нормы)': 31,
    'Активная клеточная масса, кг': 34,
    'Активная клеточная масса, кг (% от середины нормы)': 37,
    'Доля активной клеточной массы, %': 40,
    'Доля активной клеточной массы, % (% от середины нормы)': 43,
    'Скелетно-мышечная масса, кг': 46,
    'Скелетно-мышечная масса, кг (% от середины нормы)': 49,
    'Доля скелетно-мышечной массы, %': 53,
    'Доля скелетно-мышечной массы, % (% от середины нормы)': 56,
    'Основной обмен, ккал/сут.': 59,
    'Основной обмен, ккал/сут. (% от середины нормы)': 62,
    'Общая жидкость, кг': 64,
    'Общая жидкость, кг (% от середины нормы)': 67,
    'Внеклеточная жидкость, кг': 70,
    'Внеклеточная жидкость, кг (% от середины нормы)': 73,
    'Индекс талия-бедра': 75,
    'Индекс талия-бедра (% от середины нормы)': 78,
    'Доля жировой массы, %': 81,
    'Доля жировой массы, % (% от середины нормы)': 84,
    
    'Отношение внеклеточной и клеточной жидкостей': 33,
    
    'Индекс жировой массы, кг/м²': 20,
    'Индекс жировой массы, кг/м² (% от середины нормы)': 23,
    'Индекс тощей массы, кг/м²': 24,
    'Индекс тощей массы, кг/м² (% от середины нормы)': 27,
    'Индекс активной клеточной массы, кг/м²': 28,
    'Индекс активной клеточной массы, кг/м² (% от середины нормы)': 31,
    'Индекс скелетно-мышечной клеточной массы, кг/м²': 32,
    'Индекс скелетно-мышечной клеточной массы, кг/м² (% от середины нормы)': 35,
    'Минеральная масса тела, кг': 38,
    'Минеральная масса тела, кг (% от середины нормы)': 41,
    'Минеральная масса мягких тканей, кг': 44,
    'Минеральная масса мягких тканей, кг (% от середины нормы)': 47,
    'Минеральная масса костной ткани, кг': 50,
    'Минеральная масса костной ткани, кг (% от середины нормы)': 53,
    'Доля минеральной массы в ТМ, %': 56,
    'Доля минеральной массы в ТМ, % (% от середины нормы)': 59,
    'Доля минеральной массы мягких тканей в ТМ, %': 63,
    'Доля минеральной массы мягких тканей в ТМ, % (% от середины нормы)': 66,
    'Доля минеральной массы костной ткани в ТМ, %': 70,
    'Доля минеральной массы костной ткани в ТМ, % (% от середины нормы)': 73,
    'Шкала оценки риска метаболического синдрома по проценту жировой массы': 78,
    'Шкала оценки риска метаболического синдрома по проценту жировой массы (% от середины нормы)': 81
}

fns = [
]

df_bia = pd.DataFrame(columns=list(feats_rows_bia.keys()))

for fn in fns:
    print(fn)
    
    reader = PdfReader(f"{path}/{fn}.pdf")
    
    # Page 1
    page = reader.pages[0]
    lines = page.extract_text().splitlines()
    sample_id = lines[feats_rows_bia['ID']]
    df_bia.at[sample_id, 'ID'] = sample_id
    date_time = re.findall(r"([0-9]{1,2}\.[0-9]{1,2}\.[0-9]{4})\s(\d{1,2}:\d{2}:\d{2})Дата\sобследования", lines[feats_rows_bia['Дата и время обследования']])[0]
    df_bia.at[sample_id, 'Дата и время обследования'] = f"{date_time[0]} {date_time[1]}"
    df_bia.at[sample_id, 'Возраст'] = float(re.findall(r"(\d+), (.)Возраст, лет \/ Пол", lines[feats_rows_bia['Возраст']])[0][0])
    df_bia.at[sample_id, 'Пол'] = re.findall(r"(\d+), (.)Возраст, лет \/ Пол", lines[feats_rows_bia['Пол']])[0][1]
    df_bia.at[sample_id, 'Рост'] = float(re.findall(r"(\d+) \/ (\d+\.*\d*)Рост, см \/ Вес, кг", lines[feats_rows_bia['Рост']])[0][0])
    df_bia.at[sample_id, 'Вес'] = float(re.findall(r"(\d+) \/ (\d+\.*\d*)Рост, см \/ Вес, кг", lines[feats_rows_bia['Вес']])[0][1])
    df_bia.at[sample_id, 'Окр. талии, см'] = float(re.findall(r"(\d+) \/ (\d+)Окр. талии \/ Окр. бедер, см", lines[feats_rows_bia['Окр. талии, см']])[0][0])
    df_bia.at[sample_id, 'Окр. бедер, см'] = float(re.findall(r"(\d+) \/ (\d+)Окр. талии \/ Окр. бедер, см", lines[feats_rows_bia['Окр. бедер, см']])[0][1])
    df_bia.at[sample_id, 'Сопрот. акт. на 5 кГц, Ом'] = float(re.findall(r"(\d+) \/ (\d+) \/ (\d+)", lines[feats_rows_bia['Сопрот. акт. на 5 кГц, Ом']])[0][0])
    df_bia.at[sample_id, 'Сопрот. акт. на 50 кГц, Ом'] = float(re.findall(r"(\d+) \/ (\d+) \/ (\d+)", lines[feats_rows_bia['Сопрот. акт. на 50 кГц, Ом']])[0][1])
    df_bia.at[sample_id, 'Сопрот. реакт. на 50 кГц, Ом'] = float(re.findall(r"(\d+) \/ (\d+) \/ (\d+)", lines[feats_rows_bia['Сопрот. реакт. на 50 кГц, Ом']])[0][2])
    df_bia.at[sample_id, 'Фазовый угол (50 кГц), град.'] = float(lines[feats_rows_bia['Фазовый угол (50 кГц), град.']])
    df_bia.at[sample_id, 'Клеточная жидкость, кг'] = float(lines[feats_rows_bia['Клеточная жидкость, кг']])
    df_bia.at[sample_id, 'Удельный основной обмен, ккал/м²/сут.'] = float(lines[feats_rows_bia['Удельный основной обмен, ккал/м²/сут.']])
    df_bia.at[sample_id, 'Индекс массы тела'] = float(lines[feats_rows_bia['Индекс массы тела']])
    df_bia.at[sample_id, 'Индекс массы тела (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс массы тела (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Жировая масса, кг'] = float(lines[feats_rows_bia['Жировая масса, кг']])
    df_bia.at[sample_id, 'Жировая масса, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Жировая масса, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Тощая масса, кг'] = float(lines[feats_rows_bia['Тощая масса, кг']])
    df_bia.at[sample_id, 'Тощая масса, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Тощая масса, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Активная клеточная масса, кг'] = float(lines[feats_rows_bia['Активная клеточная масса, кг']])
    df_bia.at[sample_id, 'Активная клеточная масса, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Активная клеточная масса, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля активной клеточной массы, %'] = float(lines[feats_rows_bia['Доля активной клеточной массы, %']])
    df_bia.at[sample_id, 'Доля активной клеточной массы, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля активной клеточной массы, % (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Скелетно-мышечная масса, кг'] = float(lines[feats_rows_bia['Скелетно-мышечная масса, кг']])
    df_bia.at[sample_id, 'Скелетно-мышечная масса, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Скелетно-мышечная масса, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля скелетно-мышечной массы, %'] = float(lines[feats_rows_bia['Доля скелетно-мышечной массы, %']])
    df_bia.at[sample_id, 'Доля скелетно-мышечной массы, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля скелетно-мышечной массы, % (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Основной обмен, ккал/сут.'] = float(lines[feats_rows_bia['Основной обмен, ккал/сут.']])
    df_bia.at[sample_id, 'Основной обмен, ккал/сут. (% от середины нормы)'] = float(lines[feats_rows_bia['Основной обмен, ккал/сут. (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Общая жидкость, кг'] = float(lines[feats_rows_bia['Общая жидкость, кг']])
    df_bia.at[sample_id, 'Общая жидкость, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Общая жидкость, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Внеклеточная жидкость, кг'] = float(lines[feats_rows_bia['Внеклеточная жидкость, кг']])
    df_bia.at[sample_id, 'Внеклеточная жидкость, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Внеклеточная жидкость, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Индекс талия-бедра'] = float(lines[feats_rows_bia['Индекс талия-бедра']])
    df_bia.at[sample_id, 'Индекс талия-бедра (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс талия-бедра (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля жировой массы, %'] = float(lines[feats_rows_bia['Доля жировой массы, %']])
    df_bia.at[sample_id, 'Доля жировой массы, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля жировой массы, % (% от середины нормы)']].replace('%', ''))

    # Page 3
    lines = reader.pages[2].extract_text().splitlines()
    df_bia.at[sample_id, 'Отношение внеклеточной и клеточной жидкостей'] = float(re.findall(r"Ваш показатель ВКЖ\/КЖ составляет: (\d+\.*\d*) \(диапазон нормальных значений", lines[feats_rows_bia['Отношение внеклеточной и клеточной жидкостей']])[0])

    # Page 5
    lines = reader.pages[4].extract_text().splitlines()
    df_bia.at[sample_id, 'Индекс жировой массы, кг/м²'] = float(lines[feats_rows_bia['Индекс жировой массы, кг/м²']])
    df_bia.at[sample_id, 'Индекс жировой массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс жировой массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Индекс тощей массы, кг/м²'] = float(lines[feats_rows_bia['Индекс тощей массы, кг/м²']])
    df_bia.at[sample_id, 'Индекс тощей массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс тощей массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Индекс активной клеточной массы, кг/м²'] = float(lines[feats_rows_bia['Индекс активной клеточной массы, кг/м²']])
    df_bia.at[sample_id, 'Индекс активной клеточной массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс активной клеточной массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Индекс скелетно-мышечной клеточной массы, кг/м²'] = float(lines[feats_rows_bia['Индекс скелетно-мышечной клеточной массы, кг/м²']])
    df_bia.at[sample_id, 'Индекс скелетно-мышечной клеточной массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс скелетно-мышечной клеточной массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Минеральная масса тела, кг'] = float(lines[feats_rows_bia['Минеральная масса тела, кг']])
    df_bia.at[sample_id, 'Минеральная масса тела, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Минеральная масса тела, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Минеральная масса мягких тканей, кг'] = float(lines[feats_rows_bia['Минеральная масса мягких тканей, кг']])
    df_bia.at[sample_id, 'Минеральная масса мягких тканей, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Минеральная масса мягких тканей, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Минеральная масса костной ткани, кг'] = float(lines[feats_rows_bia['Минеральная масса костной ткани, кг']])
    df_bia.at[sample_id, 'Минеральная масса костной ткани, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Минеральная масса костной ткани, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля минеральной массы в ТМ, %'] = float(lines[feats_rows_bia['Доля минеральной массы в ТМ, %']])
    df_bia.at[sample_id, 'Доля минеральной массы в ТМ, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля минеральной массы в ТМ, % (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля минеральной массы мягких тканей в ТМ, %'] = float(lines[feats_rows_bia['Доля минеральной массы мягких тканей в ТМ, %']])
    df_bia.at[sample_id, 'Доля минеральной массы мягких тканей в ТМ, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля минеральной массы мягких тканей в ТМ, % (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля минеральной массы костной ткани в ТМ, %'] = float(lines[feats_rows_bia['Доля минеральной массы костной ткани в ТМ, %']])
    df_bia.at[sample_id, 'Доля минеральной массы костной ткани в ТМ, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля минеральной массы костной ткани в ТМ, % (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Шкала оценки риска метаболического синдрома по проценту жировой массы'] = float(lines[feats_rows_bia['Шкала оценки риска метаболического синдрома по проценту жировой массы']])
    df_bia.at[sample_id, 'Шкала оценки риска метаболического синдрома по проценту жировой массы (% от середины нормы)'] = float(lines[feats_rows_bia['Шкала оценки риска метаболического синдрома по проценту жировой массы (% от середины нормы)']].replace('%', ''))

df_bia = df_bia.apply(pd.to_numeric, errors='ignore')

## Different analyses 1

In [ ]:
feats_anls = {
    1: {
        'Дата и время обследования': 16,
        'Возраст': 17,
        'Пол': 7,
        'Аланинаминотрансфераза (АЛТ), Ед/л': 25,
        'Аспартатаминотрансфераза (АСТ), Ед/л': 26,
        'Витамин В9 (фолиевая кислота), нг/мл': 27,
        'Витамин В12 (цианкобаламин), пг/мл': 28,
        'Общий белок, г/л': 29,
        'Билирубин непрямой, мкмоль/л': 30,
        'Билирубин общий, мкмоль/л': 31,
        'Билирубин прямой, мкмоль/л': 32,
        'Мочевая кислота, мкмоль/л': 33,
        'Креатинин, мкмоль/л': 34,
        'Глюкоза, ммоль/л': 35,
        'Триглицериды, ммоль/л': 44,
        'ЛПОНП, ммоль/л': 50,
        'Коэффициент атерогенности': 51
    },
    2: {
        'Холестерин общий, ммоль/л': 23,
        'Холестерин липопротеидов высокой плотности (ЛПВП, HDL), ммоль/л': 37,
        'Холестерин не-ЛПВП, ммоль/л': 44,
        'Холестерин липопротеидов низкой плотности (ЛПНП, LDL), ммоль/л': 57,
        'Гомоцистеин, мкмоль/л': 65,
    },
    3: {
        'Медь, мкмоль/л': 23,
        'Магний, ммоль/л': 24,
        'Цинк, мкмоль/л': 25,
        'Железо, мкмоль/л': 26,
        'Ферритин, мкг/л': 27,
    },
    4: {
        'Тиреотропный гормон (ТТГ), мкМЕ/мл': 25,
        'Тироксин свободный (Т4 свободный), пмоль/л': 31,
        'Трийодтиронин свободный (Т3 свободный), пмоль/л': 34,
        'Антитела к тиреоглобулину (Анти-ТГ), МЕ/мл': 35,
        'Антитела к микросомальной тиреопероксидазе (Анти-ТПО), МЕ/мл': 38,
        'Фолликулостимулирующий гормон (ФСГ), мМЕ/мл': 39,
    },
    5: {
        'Лютеинизирующий гормон (ЛГ), мМЕ/мл': 23,
        'Пролактин, мМЕ/л': 40,
        'Прогестерон, нг/мл': 46,
    },
    6: {
        'Эстрадиол (Е2), пмоль/л': 23,
        'Тестостерон свободный, нмоль/л': 43,
        'Индекс свободных андрогенов, %': 44,
        'Тестостерон общий, нмоль/л': 45,
        'Глобулин, связывающий половые гормоны (ГСПГ, SHBG), нмоль/л': 48,
        'Инсулин, мкМЕ/мл': 49,
    },
    7: {
        'Кальций ионизированный, ммоль/л': 25,
    },
    8: {
        '25-OH витамин D, ИХЛА, суммарный (кальциферол), нг/мл': 27,
    },
    9: {
        'Эритроциты, 10*12/л': 25,
        'Гемоглобин, г/л': 26,
        'Гематокрит, %': 27,
        'Средний объем эритроцитов (MCV), фл': 28,
        'Среднее содержание гемоглобина в эритроците (МСН), пг/кл': 31,
        'Средняя концентрация Hb в эритроцитах (МСНС), г/дл': 34,
        'Отн.ширина распред.эритр.по объему (ст.отклонение), фл': 37,
        'Отн.ширина распред.эритр.по объему(коэфф.вариации), %': 40,
        'Тромбоциты, 10*9/л': 41,
        'Средний объем тромбоцитов (MPV), фл': 42,
        'Тромбокрит (PCT), %': 43,
        'Относит.ширина распред.тромбоцитов по объему (PDW), %': 46,
        'Лейкоциты, 10*9/л': 47,
        'Нейтрофилы, 10*9/л': 48,
        'Нейтрофилы, %': 49,
        'Эозинофилы, 10*9/л': 50,
        'Эозинофилы, %': 51,
        'Базофилы, 10*9/л': 52,
        'Базофилы, %': 53,
        'Моноциты, 10*9/л': 54,
        'Моноциты, %': 55,
        'Лимфоциты, 10*9/л': 56,
        'Лимфоциты, %': 57,
    },
    10: {
        'Гликированный гемоглобин А1с, %': 25,
    }
}

cols_anls = ['ID'] + list(itertools.chain.from_iterable([list(feats_anls[x].keys()) for x in feats_anls])) 
df_anls = pd.DataFrame(columns=list(cols_anls))

fns = [
]

for fn in fns:
    print(fn)
    
    reader = PdfReader(f"{path}/{fn}.pdf")

    for page in reader.pages:
        lines = page.extract_text().splitlines()
        sample_id = lines[11].capitalize() + ' ' + re.findall(r"(.*)Имя", lines[10])[0]
        if lines[25].startswith('Аланинаминотрансфераза (АЛТ)'):
            page_id = 1
            df_anls.at[sample_id, 'ID'] = sample_id
            df_anls.at[sample_id, 'Дата и время обследования'] = lines[feats_anls[page_id]['Дата и время обследования']]
            df_anls.at[sample_id, 'Возраст'] = float(re.findall(r"\((\d+) .\)", lines[feats_anls[page_id]['Возраст']])[0])
            df_anls.at[sample_id, 'Пол'] = re.findall(r"Пол: (.+)", lines[feats_anls[page_id]['Пол']])[0][0]
            df_anls.at[sample_id, 'Аланинаминотрансфераза (АЛТ), Ед/л'] = re.findall(r"Аланинаминотрансфераза \(АЛТ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Аланинаминотрансфераза (АЛТ), Ед/л']])[0]
            df_anls.at[sample_id, 'Аспартатаминотрансфераза (АСТ), Ед/л'] = re.findall(r"Аспартатаминотрансфераза \(АСТ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Аспартатаминотрансфераза (АСТ), Ед/л']])[0]
            df_anls.at[sample_id, 'Витамин В9 (фолиевая кислота), нг/мл'] = re.findall(r"Витамин В9 \(фолиевая кислота\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Витамин В9 (фолиевая кислота), нг/мл']])[0]
            df_anls.at[sample_id, 'Витамин В12 (цианкобаламин), пг/мл'] = re.findall(r"Витамин В12 \(цианкобаламин\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Витамин В12 (цианкобаламин), пг/мл']])[0]
            df_anls.at[sample_id, 'Общий белок, г/л'] = re.findall(r"Общий белок (\d+\.*\d*) .*", lines[feats_anls[page_id]['Общий белок, г/л']])[0]
            df_anls.at[sample_id, 'Билирубин непрямой, мкмоль/л'] = re.findall(r"Билирубин непрямой (\d+\.*\d*) .*", lines[feats_anls[page_id]['Билирубин непрямой, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Билирубин общий, мкмоль/л'] = re.findall(r"Билирубин общий (\d+\.*\d*) .*", lines[feats_anls[page_id]['Билирубин общий, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Билирубин прямой, мкмоль/л'] = re.findall(r"Билирубин прямой (\d+\.*\d*) .*", lines[feats_anls[page_id]['Билирубин прямой, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Мочевая кислота, мкмоль/л'] = re.findall(r"Мочевая кислота (\d+\.*\d*) .*", lines[feats_anls[page_id]['Мочевая кислота, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Креатинин, мкмоль/л'] = re.findall(r"Креатинин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Креатинин, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Глюкоза, ммоль/л'] = re.findall(r"Глюкоза (\d+\.*\d*) .*", lines[feats_anls[page_id]['Глюкоза, ммоль/л']])[0]
            df_anls.at[sample_id, 'Триглицериды, ммоль/л'] = re.findall(r"Триглицериды (\d+\.*\d*) .*", lines[feats_anls[page_id]['Триглицериды, ммоль/л']])[0]
            df_anls.at[sample_id, 'ЛПОНП, ммоль/л'] = re.findall(r"ЛПОНП (\d+\.*\d*) .*", lines[feats_anls[page_id]['ЛПОНП, ммоль/л']])[0]
            df_anls.at[sample_id, 'Коэффициент атерогенности'] = re.findall(r"Коэффициент атерогенности (\d+\.*\d*) .*", lines[feats_anls[page_id]['Коэффициент атерогенности']])[0]
        elif lines[23].startswith('Холестерин общий'):
            page_id = 2
            df_anls.at[sample_id, 'Холестерин общий, ммоль/л'] = re.findall(r"Холестерин общий (\d+\.*\d*) .*", lines[feats_anls[page_id]['Холестерин общий, ммоль/л']])[0]
            df_anls.at[sample_id, 'Холестерин липопротеидов высокой плотности (ЛПВП, HDL), ммоль/л'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Холестерин липопротеидов высокой плотности (ЛПВП, HDL), ммоль/л']])[0]
            df_anls.at[sample_id, 'Холестерин не-ЛПВП, ммоль/л'] = re.findall(r"Холестерин не-ЛПВП (\d+\.*\d*) .*", lines[feats_anls[page_id]['Холестерин не-ЛПВП, ммоль/л']])[0]
            df_anls.at[sample_id, 'Холестерин липопротеидов низкой плотности (ЛПНП, LDL), ммоль/л'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Холестерин липопротеидов низкой плотности (ЛПНП, LDL), ммоль/л']])[0]
            df_anls.at[sample_id, 'Гомоцистеин, мкмоль/л'] = re.findall(r"Гомоцистеин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Гомоцистеин, мкмоль/л']])[0]
        elif lines[23].startswith('Медь'):
            page_id = 3
            df_anls.at[sample_id, 'Медь, мкмоль/л'] = re.findall(r"Медь (\d+\.*\d*) .*", lines[feats_anls[page_id]['Медь, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Магний, ммоль/л'] = re.findall(r"Магний (\d+\.*\d*) .*", lines[feats_anls[page_id]['Магний, ммоль/л']])[0]
            df_anls.at[sample_id, 'Цинк, мкмоль/л'] = re.findall(r"Цинк (\d+\.*\d*) .*", lines[feats_anls[page_id]['Цинк, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Железо, мкмоль/л'] = re.findall(r"Железо (\d+\.*\d*) .*", lines[feats_anls[page_id]['Железо, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Ферритин, мкг/л'] = re.findall(r"Ферритин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Ферритин, мкг/л']])[0]
        elif lines[24].startswith('ГОРМОНЫ'):
            page_id = 4
            df_anls.at[sample_id, 'Тиреотропный гормон (ТТГ), мкМЕ/мл'] = re.findall(r"Тиреотропный гормон \(ТТГ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тиреотропный гормон (ТТГ), мкМЕ/мл']])[0]
            df_anls.at[sample_id, 'Тироксин свободный (Т4 свободный), пмоль/л'] = re.findall(r"Тироксин свободный \(Т4 свободный\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тироксин свободный (Т4 свободный), пмоль/л']])[0]
            df_anls.at[sample_id, 'Трийодтиронин свободный (Т3 свободный), пмоль/л'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Трийодтиронин свободный (Т3 свободный), пмоль/л']])[0]
            df_anls.at[sample_id, 'Антитела к тиреоглобулину (Анти-ТГ), МЕ/мл'] = re.findall(r"Антитела к тиреоглобулину \(Анти-ТГ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Антитела к тиреоглобулину (Анти-ТГ), МЕ/мл']])[0]
            df_anls.at[sample_id, 'Антитела к микросомальной тиреопероксидазе (Анти-ТПО), МЕ/мл'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Антитела к микросомальной тиреопероксидазе (Анти-ТПО), МЕ/мл']])[0]
            df_anls.at[sample_id, 'Фолликулостимулирующий гормон (ФСГ), мМЕ/мл'] = re.findall(r"Фолликулостимулирующий гормон \(ФСГ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Фолликулостимулирующий гормон (ФСГ), мМЕ/мл']])[0]
        elif lines[23].startswith('Лютеинизирующий гормон (ЛГ)'):
            page_id = 5
            df_anls.at[sample_id, 'Лютеинизирующий гормон (ЛГ), мМЕ/мл'] = re.findall(r"Лютеинизирующий гормон \(ЛГ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Лютеинизирующий гормон (ЛГ), мМЕ/мл']])[0]
            df_anls.at[sample_id, 'Пролактин, мМЕ/л'] = re.findall(r"Пролактин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Пролактин, мМЕ/л']])[0]
            df_anls.at[sample_id, 'Прогестерон, нг/мл'] = re.findall(r"Прогестерон (\<*\d+\.*\d*) .*", lines[feats_anls[page_id]['Прогестерон, нг/мл']])[0]
        elif lines[23].startswith('Эстрадиол (Е2)'):
            page_id = 6
            df_anls.at[sample_id, 'Эстрадиол (Е2), пмоль/л'] = re.findall(r"Эстрадиол \(Е2\) (\<*\d+\.*\d*) .*", lines[feats_anls[page_id]['Эстрадиол (Е2), пмоль/л']].replace('\u2009', ' '))[0]
            df_anls.at[sample_id, 'Тестостерон свободный, нмоль/л'] = re.findall(r"Тестостерон свободный (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тестостерон свободный, нмоль/л']])[0]
            df_anls.at[sample_id, 'Индекс свободных андрогенов, %'] = re.findall(r"Индекс свободных андрогенов (\d+\.*\d*) .*", lines[feats_anls[page_id]['Индекс свободных андрогенов, %']])[0]
            df_anls.at[sample_id, 'Тестостерон общий, нмоль/л'] = re.findall(r"Тестостерон общий (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тестостерон общий, нмоль/л']])[0]
            df_anls.at[sample_id, 'Глобулин, связывающий половые гормоны (ГСПГ, SHBG), нмоль/л'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Глобулин, связывающий половые гормоны (ГСПГ, SHBG), нмоль/л']])[0]
            df_anls.at[sample_id, 'Инсулин, мкМЕ/мл'] = re.findall(r"Инсулин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Инсулин, мкМЕ/мл']])[0]
        elif lines[25].startswith('Кальций ионизированный'):
            page_id = 7
            df_anls.at[sample_id, 'Кальций ионизированный, ммоль/л'] = re.findall(r"Кальций ионизированный (\d+\.*\d*) .*", lines[feats_anls[page_id]['Кальций ионизированный, ммоль/л']])[0]
        elif lines[24].startswith('ВИТАМИНЫ'):
            page_id = 8
            df_anls.at[sample_id, '25-OH витамин D, ИХЛА, суммарный (кальциферол), нг/мл'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['25-OH витамин D, ИХЛА, суммарный (кальциферол), нг/мл']])[0]
        elif lines[24].startswith('ГЕМАТОЛОГИЧЕСКИЕ ИССЛЕДОВАНИЯ'):
            page_id = 9
            df_anls.at[sample_id, 'Эритроциты, 10*12/л'] = re.findall(r"Эритроциты (\d+\.*\d*) .*", lines[feats_anls[page_id]['Эритроциты, 10*12/л']])[0]
            df_anls.at[sample_id, 'Гемоглобин, г/л'] = re.findall(r"Гемоглобин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Гемоглобин, г/л']])[0]
            df_anls.at[sample_id, 'Гематокрит, %'] = re.findall(r"Гематокрит (\d+\.*\d*) .*", lines[feats_anls[page_id]['Гематокрит, %']])[0]
            df_anls.at[sample_id, 'Средний объем эритроцитов (MCV), фл'] = re.findall(r"Средний объем эритроцитов \(MCV\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Средний объем эритроцитов (MCV), фл']])[0]
            df_anls.at[sample_id, 'Среднее содержание гемоглобина в эритроците (МСН), пг/кл'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Среднее содержание гемоглобина в эритроците (МСН), пг/кл']])[0]
            df_anls.at[sample_id, 'Средняя концентрация Hb в эритроцитах (МСНС), г/дл'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Средняя концентрация Hb в эритроцитах (МСНС), г/дл']])[0]
            df_anls.at[sample_id, 'Отн.ширина распред.эритр.по объему (ст.отклонение), фл'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Отн.ширина распред.эритр.по объему (ст.отклонение), фл']])[0]
            df_anls.at[sample_id, 'Отн.ширина распред.эритр.по объему(коэфф.вариации), %'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Отн.ширина распред.эритр.по объему(коэфф.вариации), %']])[0]
            df_anls.at[sample_id, 'Тромбоциты, 10*9/л'] = re.findall(r"Тромбоциты (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тромбоциты, 10*9/л']])[0]
            df_anls.at[sample_id, 'Средний объем тромбоцитов (MPV), фл'] = re.findall(r"Средний объем тромбоцитов \(MPV\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Средний объем тромбоцитов (MPV), фл']])[0]
            df_anls.at[sample_id, 'Тромбокрит (PCT), %'] = re.findall(r"Тромбокрит \(PCT\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тромбокрит (PCT), %']])[0]
            df_anls.at[sample_id, 'Относит.ширина распред.тромбоцитов по объему (PDW), %'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Относит.ширина распред.тромбоцитов по объему (PDW), %']])[0]
            df_anls.at[sample_id, 'Лейкоциты, 10*9/л'] = re.findall(r"Лейкоциты (\d+\.*\d*) .*", lines[feats_anls[page_id]['Лейкоциты, 10*9/л']])[0]
            df_anls.at[sample_id, 'Нейтрофилы, 10*9/л'] = re.findall(r"Нейтрофилы (\d+\.*\d*) .*", lines[feats_anls[page_id]['Нейтрофилы, 10*9/л']])[0]
            df_anls.at[sample_id, 'Нейтрофилы, %'] = re.findall(r"Нейтрофилы \% (\d+\.*\d*) .*", lines[feats_anls[page_id]['Нейтрофилы, %']])[0]
            df_anls.at[sample_id, 'Эозинофилы, 10*9/л'] = re.findall(r"Эозинофилы (\d+\.*\d*) .*", lines[feats_anls[page_id]['Эозинофилы, 10*9/л']])[0]
            df_anls.at[sample_id, 'Эозинофилы, %'] = re.findall(r"Эозинофилы \% (\d+\.*\d*) .*", lines[feats_anls[page_id]['Эозинофилы, %']])[0]
            df_anls.at[sample_id, 'Базофилы, 10*9/л'] = re.findall(r"Базофилы (\d+\.*\d*) .*", lines[feats_anls[page_id]['Базофилы, 10*9/л']])[0]
            df_anls.at[sample_id, 'Базофилы, %'] = re.findall(r"Базофилы \% (\d+\.*\d*) .*", lines[feats_anls[page_id]['Базофилы, %']])[0]
            df_anls.at[sample_id, 'Моноциты, 10*9/л'] = re.findall(r"Моноциты (\d+\.*\d*) .*", lines[feats_anls[page_id]['Моноциты, 10*9/л']])[0]
            df_anls.at[sample_id, 'Моноциты, %'] = re.findall(r"Моноциты \% (\d+\.*\d*) .*", lines[feats_anls[page_id]['Моноциты, %']])[0]
            df_anls.at[sample_id, 'Лимфоциты, 10*9/л'] = re.findall(r"Лимфоциты (\d+\.*\d*) .*", lines[feats_anls[page_id]['Лимфоциты, 10*9/л']])[0]
            df_anls.at[sample_id, 'Лимфоциты, %'] = re.findall(r"Лимфоциты \% (\d+\.*\d*) .*", lines[feats_anls[page_id]['Лимфоциты, %']])[0]
        elif lines[25].startswith('Гликированный гемоглобин А1с'):
            page_id = 10
            df_anls.at[sample_id, 'Гликированный гемоглобин А1с, %'] = re.findall(r"Гликированный гемоглобин А1с (\d+\.*\d*) .*", lines[feats_anls[page_id]['Гликированный гемоглобин А1с, %']])[0]
    
df_anls = df_anls.apply(pd.to_numeric, errors='ignore')


## Electrocardiogram

In [ ]:
feats_rows_ecg = {
    'ID': 11,
    'Дата и время обследования': 22,
    'Возраст': 14,
    'Пол': 13,
    'Длительность P, мс': 32,
    'Длительность QRS, мс': 41,
    'Интервал PQ, мс': 31,
    'Интервал QT, мс': 40,
    'Интервал QTc, мс': 47,
    'Ось P': 30,
    'Ось QRS': 39,
    'Ось T': 46,
    'ЧСС, уд/мин': 25,
    'Нерегулярность ритма, %': 51
}

fns = [
]

df_ecg = pd.DataFrame(columns=list(feats_rows_ecg.keys()))

for fn in fns:
    print(fn)
    
    reader = PdfReader(f"{path}/{fn}.pdf")
    
    # Page 1
    page = reader.pages[0]
    lines = page.extract_text().splitlines()
    if lines[16] == 'Дата/время:':
        row_id_inc = -1
    else:
        row_id_inc = 0
    sample_id = lines[feats_rows_ecg['ID']]
    df_ecg.at[sample_id, 'ID'] = sample_id
    df_ecg.at[sample_id, 'Дата и время обследования'] = lines[feats_rows_ecg['Дата и время обследования'] + row_id_inc]
    df_ecg.at[sample_id, 'Возраст'] = float(re.findall(r"(\d+) .*", lines[feats_rows_ecg['Возраст']])[0])
    df_ecg.at[sample_id, 'Пол'] = lines[feats_rows_ecg['Пол']][0].upper()
    df_ecg.at[sample_id, 'Длительность P, мс'] = re.findall(r"\s*(\d+)\s*мс", lines[feats_rows_ecg['Длительность P, мс'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Длительность QRS, мс'] = re.findall(r"\s*(\d+)\s*мс", lines[feats_rows_ecg['Длительность QRS, мс'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Интервал PQ, мс'] = re.findall(r"\s*(\d+)\s*мс", lines[feats_rows_ecg['Интервал PQ, мс'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Интервал QT, мс'] = re.findall(r"\s*(\d+)\s*мс", lines[feats_rows_ecg['Интервал QT, мс'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Интервал QTc, мс'] = re.findall(r"\s*(\d+)\s*мс", lines[feats_rows_ecg['Интервал QTc, мс'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Ось P'] = re.findall(r"\s*(-?[0-9]\d*(\.\d+)?)", lines[feats_rows_ecg['Ось P'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Ось QRS'] = re.findall(r"\s*(-?[0-9]\d*)", lines[feats_rows_ecg['Ось QRS'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Ось T'] = re.findall(r"\s*(-?[0-9]\d*)", lines[feats_rows_ecg['Ось T'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'ЧСС, уд/мин'] = re.findall(r"\s*(\d+)\s*уд\/мин", lines[feats_rows_ecg['ЧСС, уд/мин'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Нерегулярность ритма, %'] = re.findall(r"\s+Нерегулярность\s+ритма\s+=\s+(\d+)\%", lines[feats_rows_ecg['Нерегулярность ритма, %'] + row_id_inc])[0]
    
df_ecg = df_ecg.apply(pd.to_numeric, errors='ignore')

## Different analyses 2

In [ ]:
fns = [
]

feats = pd.read_excel(f"{path}/feats.xlsx", index_col=0)
feats_dict = dict(zip(feats.index.values, feats['feature'].values))

df_anls = pd.DataFrame(columns=['ID', 'Возраст', 'Пол', 'Дата и время'] + list(feats['feature']))
missed_lines = []

for fn in fns:
    print(fn)
    
    reader = PdfReader(f"{path}/{fn}.pdf")

    lines = []
    for page in reader.pages:
        lines += page.extract_text().splitlines()

    sample_info = re.findall(r"Пациент: (.+), (\d+) .+, (.+)", lines[3])[0]
    date_and_time = re.findall(r"Дата приёма: (.*)", lines[1])[0]
    sample_id = f"{sample_info[0]} {date_and_time}"
    df_anls.at[sample_id, 'ID'] = sample_info[0]
    print(sample_id)
    df_anls.at[sample_id, 'Возраст'] = sample_info[1]
    df_anls.at[sample_id, 'Пол'] = sample_info[2][0].upper()
    df_anls.at[sample_id, 'Дата и время'] = date_and_time

    for line in lines[7::]:
        line_parse = re.findall(fr"(.*): ([-+]?(?:\d+\.\d+|\d+|\.\d+))\s*", line)
        if len(line_parse) > 0:
            if line_parse[0][0] in feats_dict:
                df_anls.at[sample_id, feats_dict[line_parse[0][0]]] = line_parse[0][1]
            else:
                missed_lines.append(line)
        else:
            missed_lines.append(line)

df_anls = df_anls.apply(pd.to_numeric, errors='ignore')

In [ ]:
line = lines[31]
line_parse = re.findall(fr"(.*): ([-+]?(?:\d+\.\d+|\d+|\.\d+))\s*", line)[0]

In [ ]:
# extract only text oriented up
print(page.extract_text(0))

# extract text oriented up and turned left
print(page.extract_text((0, 90)))

# extract text in a fixed width format that closely adheres to the rendered
# layout in the source pdf
print(page.extract_text(extraction_mode="layout"))

# extract text preserving horizontal positioning without excess vertical
# whitespace (removes blank and "whitespace only" lines)
print(page.extract_text(extraction_mode="layout", layout_mode_space_vertically=False))

# adjust horizontal spacing
print(page.extract_text(extraction_mode="layout", layout_mode_scale_weight=1.0))

# exclude (default) or include (as shown below) text rotated w.r.t. the page
print(page.extract_text(extraction_mode="layout", layout_mode_strip_rotated=False))

In [ ]:
reader = PdfReader(f"{path}/Бурминская Светлана Александровна, #2, 2024-10-11 10_55_46.pdf")
page = reader.pages[0]
print(page.extract_text(0))

In [ ]:
print(page.extract_text(extraction_mode="layout", layout_mode_space_vertically=False))

In [ ]:
import re

def extract_floats(text):
    # Паттерн для поиска чисел с плавающей точкой в указанном формате
    pattern = r":\s*(\d+\.\d+)\s*кг"
    # Находим все совпадения в тексте
    matches = re.findall(pattern, text)
    # Конвертируем найденные строки в числа типа float
    return [float(match) for match in matches]

# Пример использования
text = """
Ваша активная клеточная масса составляет: 24.1 кг (диапазон)
Другие показатели: жировая масса 15.5 кг, вода: 30.0 кг.
Еще один пример: 42.8 кг (невероятный результат)!
"""

floats = extract_floats(text)
print("Найденные числа:", floats)

In [ ]:
import re

text = "39, ЖВозраст, лет / Пол"
result = re.findall(r"(\d+), (\S)Возраст.*", text)